# Transforming

Now we consider transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

## Setup

In [15]:
import openai
import os

from IPython.display import display, HTML
from redlines import Redlines
from IPython.display import Markdown

from dotenv import load_dotenv
load_dotenv(".env")

openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

## Translation

In [6]:
prompt = f"""
Translation the following English text to German. \
Use polite present form. \
`
Hi, I would like to order a blender.
`
"""
print(get_completion(prompt))

Guten Tag, ich möchte gerne einen Mixer bestellen.


In [5]:
prompt = f"""
Was ist der Unterschied zwischen 'würde gerne' und 'hätte gerne'?
"""
print(get_completion(prompt))

"Würde gerne" drückt den Wunsch aus, etwas in der Zukunft zu tun oder zu haben, während "hätte gerne" den Wunsch ausdrückt, etwas in der Gegenwart zu haben oder zu tun, das im Moment nicht verfügbar ist. Zum Beispiel: "Ich würde gerne nächstes Wochenende ins Kino gehen" (Zukunft) vs. "Ich hätte gerne ein Stück Kuchen" (Gegenwart).


In [7]:
prompt = f"""
Tell me which language this is:
`Combien coûte le lampadaire?`
"""
print(get_completion(prompt))

This is French.


In [8]:
prompt = f"""
Translate the following  text to German and Spanish
and English pirate: \
```I want to order a basketball```
"""
response = get_completion(prompt)
print(response)

German: Ich möchte einen Basketball bestellen.
Spanish: Quiero pedir una pelota de baloncesto.
English pirate: Arrr, I be wantin' to order a basketball.


In [10]:
prompt = f"""
Translate the following text to German in formal and informal form:
`
Are you take off the bus?
`
"""
print(get_completion(prompt))

Formal: Steigen Sie aus dem Bus aus?
Informal: Steigst du aus dem Bus aus?


## Universal Translator

In [11]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
]

In [12]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (This is French.): La performance du système est plus lente que d'habitude.
English: The system performance is slower than usual.
Korean: 시스템 성능이 평소보다 느립니다. 

Original message (This is Spanish.): Mi monitor tiene píxeles que no se iluminan.
English: My monitor has pixels that don't light up.
Korean: 내 모니터에는 불이 켜지지 않는 픽셀이 있습니다. 

Original message (This is Italian.): Il mio mouse non funziona
English: My mouse is not working.
Korean: 내 마우스가 작동하지 않습니다. 

Original message (This is Polish.): Mój klawisz Ctrl jest zepsuty
English: My Ctrl key is broken.
Korean: 제 Ctrl 키가 고장 났어요. 

Original message (This is Chinese (Simplified).): 我的屏幕在闪烁
English: My screen is flickering.
Korean: 내 화면이 깜빡입니다. 



## Tone Transformation

In [13]:
prompt = f"""
Translate the following text from slang to business setting:
`
Dude, what's up? This is Joe, check out this spec on this standing lamp.
`
"""
print(get_completion(prompt))

Hello, how are you? This is Joe. Please take a look at the specifications for this standing lamp.


## Format Conversion

In [14]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following Python dictionary to an HTML table \
with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)
display(HTML(response))

<table>
  <caption>Resturant Employees</caption>
  <thead>
    <tr>
      <th>Name</th>
      <th>Email</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Shyam</td>
      <td>shyamjaiswal@gmail.com</td>
    </tr>
    <tr>
      <td>Bob</td>
      <td>bob32@gmail.com</td>
    </tr>
    <tr>
      <td>Jai</td>
      <td>jai87@gmail.com</td>
    </tr>
  </tbody>
</table>


Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## Spellcheck/Grammar check

In [15]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

The girl with the black and white puppies has a ball.
No errors found.
It's going to be a long day. Does the car need its oil changed?
Their goes my freedom. There going to bring they're suitcases.

Corrected version: 
There goes my freedom. They're going to bring their suitcases.
You're going to need your notebook.
That medicine affects my ability to sleep. Have you heard of the butterfly effect?
This phrase is to check ChatGPT for spelling ability.


In [18]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```"
response = get_completion(prompt)
print(response)

I got this for my daughter's birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. However, one of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. Additionally, it's a bit small for what I paid for it. I think there might be other options that are bigger for the same price. On the positive side, it arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [14]:
diff = Redlines(text, response)
display(Markdown(diff.output_markdown))

<span style="color:red;font-weight:700;text-decoration:line-through;">Got </span><span style="color:red;font-weight:700;">I got </span>this for my <span style="color:red;font-weight:700;text-decoration:line-through;">daughter for her </span><span style="color:red;font-weight:700;">daughter's </span>birthday <span style="color:red;font-weight:700;text-decoration:line-through;">cuz </span><span style="color:red;font-weight:700;">because </span>she keeps taking mine from my <span style="color:red;font-weight:700;text-decoration:line-through;">room.  </span><span style="color:red;font-weight:700;">room. </span>Yes, adults also like pandas <span style="color:red;font-weight:700;text-decoration:line-through;">too.  </span><span style="color:red;font-weight:700;">too. </span>She takes it everywhere with her, and it's super soft and <span style="color:red;font-weight:700;text-decoration:line-through;">cute.  One </span><span style="color:red;font-weight:700;">cute. However, one </span>of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. <span style="color:red;font-weight:700;text-decoration:line-through;">It's </span><span style="color:red;font-weight:700;">Additionally, it's </span>a bit small for what I paid for <span style="color:red;font-weight:700;text-decoration:line-through;">it though. </span><span style="color:red;font-weight:700;">it. </span>I think there might be other options that are bigger for the same <span style="color:red;font-weight:700;text-decoration:line-through;">price.  It </span><span style="color:red;font-weight:700;">price. On the positive side, it </span>arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

In [19]:
prompt = f"""
proofread and correct this review.
Make it more compelling.
Ensure it follows APA style guide and targets
an advanced reader.
Output in Markdown format.
Text: `{text}`.
"""
response = get_completion(prompt)
display(Markdown(response))

I purchased this adorable panda plush toy for my daughter's birthday, as she has a habit of taking mine from my room. Despite being an adult, I couldn't resist the cuteness of this toy myself. My daughter has taken it everywhere with her since receiving it, and it's incredibly soft to the touch. However, I did notice that one of the ears is slightly lower than the other, which I don't believe was intentional. Additionally, I found the size to be a bit smaller than expected for the price I paid. It's possible that there are larger options available for the same cost. Despite these minor issues, I was pleased to receive the toy a day earlier than expected, which allowed me to enjoy playing with it before giving it to my daughter. Overall, I would recommend this panda plush toy to anyone who loves cute and cuddly companions. 

APA Style:
I purchased an adorable panda plush toy for my daughter's birthday, as she has a habit of taking mine from my room. Despite being an adult, I couldn't resist the cuteness of this toy myself. My daughter has taken it everywhere with her since receiving it, and it's incredibly soft to the touch. However, I did notice that one of the ears is slightly lower than the other, which I don't believe was intentional. Additionally, I found the size to be a bit smaller than expected for the price I paid. It's possible that there are larger options available for the same cost. Despite these minor issues, I was pleased to receive the toy a day earlier than expected, which allowed me to enjoy playing with it before giving it to my daughter. Overall, I would recommend this panda plush toy to anyone who loves cute and cuddly companions.

In [20]:
text

"\nGot this for my daughter for her birthday cuz she keeps taking mine from my room.  Yes, adults also like pandas too.  She takes it everywhere with her, and it's super soft and cute.  One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price.  It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.\n"